In [0]:
#(1)ドライブをマウント
#(2)学習済みモデルのあるディレクトリに移動

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd drive

/content/drive


In [4]:
cd My Drive

/content/drive/My Drive


In [0]:
import torch
import torchvision
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
from PIL import Image
from torchvision import models, transforms
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [0]:
from torchvision import datasets

In [7]:
!pip install pytorch-lightning

     |████████████████████████████████| 204kB 32.2MB/s 
     |████████████████████████████████| 829kB 42.9MB/s 
     |████████████████████████████████| 61kB 9.6MB/s 
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=b7bd043d42d4844d8240d287ace283c01ac9eea02115814c6bdd42551bc4bb4d
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0
  Found existing installation: tqdm 4.38.0
    Uninstalling tqdm-4.38.0:
      Successfully uninstalled tqdm-4.38.0


In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_pretrained = True  # 学習済みのパラメータを使用
model = models.densenet161(pretrained=use_pretrained).to(device)

Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/checkpoints/densenet161-8d451a50.pth


In [0]:
model.classifier = nn.Linear(in_features=2208, out_features=10, bias=True).to(device)

In [13]:
#モデルを読み込む
device = torch.device("cuda")
#model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load("check.pt"))
model.to(device)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel

In [0]:
transform_test = transforms.Compose([
    transforms.RandomResizedCrop(224), #サイズ合わせ
    transforms.RandomHorizontalFlip(), 
    #transforms.RandomVerticalFlip(),
    transforms.ToTensor() ,              #型変換(?)
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  #https://teratail.com/questions/234027
    
    ])

In [0]:
your_datasets = datasets.ImageFolder(root="cholab", transform=transform_test) 

In [0]:
loader = torch.utils.data.DataLoader(your_datasets, batch_size=2)

In [85]:
for batch_index, (X, y) in enumerate(loader):
  type(X)
  type(y)

type(X)
type(y)

torch.Tensor

In [0]:
X = X.cuda()

In [87]:
X.shape

torch.Size([2, 3, 224, 224])

In [0]:
# 評価モードにする
model = model.eval()

In [89]:
y_predict = model.forward(X)
y_predict

tensor([[-2.6050, -0.0767,  0.3529,  9.6761, -2.5028,  1.0812, -3.5540,  0.5099,
         -1.7908, -0.1376],
        [-2.1664, -1.0139, -0.0816,  7.4182, -2.0700,  0.4961, -2.4589,  0.6900,
         -0.9406,  0.2020]], device='cuda:0', grad_fn=<AddmmBackward>)

In [90]:
y_predict[0]

tensor([-2.6050, -0.0767,  0.3529,  9.6761, -2.5028,  1.0812, -3.5540,  0.5099,
        -1.7908, -0.1376], device='cuda:0', grad_fn=<SelectBackward>)

In [68]:
y_predict[1]

tensor([-2.2841, -1.2309, -0.1717,  7.0933, -1.7402,  0.5590, -2.6944,  0.7980,
        -0.4866,  0.6529], device='cuda:0', grad_fn=<SelectBackward>)

In [70]:
y_predict[0].argmax()

tensor(3, device='cuda:0')

In [71]:
y_predict[1].argmax()

tensor(3, device='cuda:0')

In [0]:
class_name = ["airplane","bird","car","cat","deer","dog","horse","monkey","shiptuck"]

In [72]:
class_name[y_predict[0].argmax()]

'cat'

In [73]:
class_name[y_predict[1].argmax()]

'cat'